In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt','r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
chars = sorted(list(set(''.join(words))))
stoi =  {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
# build the dataset

block_size = 3  # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:

    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


In [7]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [27]:
C = torch.randn((27,2))

In [36]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [30]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [59]:
h = torch.tanh(emb.view(-1 ,6) @ W1 + b1) 

In [61]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [62]:
logits = h @ W2 + b2

In [63]:
logits.shape

torch.Size([32, 27])

In [64]:
counts = logits.exp()

In [65]:
prob = counts/counts.sum(1, keepdims=True)

In [75]:
loss = -prob[torch.arange(32),Y].log().mean()

tensor(12.5782)

In [76]:
# ---------------------------- ORDER -------------------------------------

In [77]:
X.shape, Y.shape # dataset

(torch.Size([32, 3]), torch.Size([32]))

In [81]:
# parameters
g = torch.Generator().manual_seed(214783647)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6,100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100,27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [83]:
sum(p.nelement() for p in parameters)

3481

In [85]:
# forward pass
emb = C[X] # (32, 3, 2)
h = torch.tanh(emb.view(-1 ,6) @ W1 + b1) 
logits = h @ W2 + b2
#counts = logits.exp()
#prob = counts/counts.sum(1, keepdims=True)
loss = F.cross_entropy(logits,Y)
loss

tensor(17.8873)

tensor(17.8873)